# Process model prediction

In [53]:
import glob
import numpy as np
import xarray as xr

import tensorflow as tf

## Open dataset for prediction

In [3]:
path_file = "../data/validate/2019_04"#_ERA5.nc"
ds = xr.open_dataset(path_file)

In [40]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 21, longitude: 21, time: 720)
Coordinates:
  * longitude  (longitude) float32 -125.0 -124.75 -124.5 ... -120.25 -120.0
  * latitude   (latitude) float32 40.0 39.75 39.5 39.25 ... 35.5 35.25 35.0
  * time       (time) datetime64[ns] 2019-04-01 ... 2019-04-30T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-06-08 22:08:43 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

We are going to manually recreate the functionality of the ```slice_generator``` class here,
just to pull out an input for the model

In [9]:
# Choose arbitrary slice
start = 427
end   = 429

In [22]:
# Extract slice and process
array = ds['t2m'].isel(time=slice(start, end)).values
print(array.shape)
array = np.moveaxis(array, 0, 1)
print(array.shape)
array = array.reshape(-1, 2, 1, 21, 21)
print(array.shape)

(2, 21, 21)
(21, 2, 21)
(1, 2, 1, 21, 21)


## Import model and predict

In [54]:
model_dir = "../models/"
models_list = sorted(glob.glob(model_dir+"*.h5"))
print(models_list)

['../models/full_stack_1f_1c_21x_21y.h5', '../models/t_full_stack_1f_1c_21x_21y.h5']


In [55]:
# choose a model
file_index = 1
models_list[file_index]
model_name = 'full_stack_1f_1c_21x_21y' # todo: use regex to parse filename

model = tf.keras.models.load_model(models_list[file_index])
model.summary()

Model: "Full_stack"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_input (InputLayer)        [(None, 2, 1, 21, 21 0                                            
__________________________________________________________________________________________________
tf_op_layer_unstack_1 (TensorFl [(None, 1, 21, 21),  0           model_input[0][0]                
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, 1, 21, 21)    0           tf_op_layer_unstack_1[0][0]      
__________________________________________________________________________________________________
convB1 (Conv2D)                 (None, 8, 9, 9)      208         tf_op_layer_unstack_1[0][1]      
_________________________________________________________________________________________

In [56]:
pred = model.predict(array, verbose=1)
pred

1/1 [==============================] - 0s 2ms/step


array([[[[[90.887764, 90.887764, 90.887764, 91.39598 , 91.51119 ,
           90.887764, 90.887764, 90.887764, 93.77147 , 90.887764,
           91.389565, 90.887764, 90.887764, 90.887764, 92.01836 ,
           90.887764, 90.887764, 90.887764, 91.58931 , 90.887764,
           90.887764],
          [90.887764, 90.887764, 91.75855 , 90.887764, 90.887764,
           90.887764, 90.887764, 90.887764, 90.887764, 94.156   ,
           90.887764, 92.41626 , 90.887764, 90.887764, 90.887764,
           90.887764, 90.887764, 92.00379 , 90.887764, 90.887764,
           90.887764],
          [99.02188 , 95.13313 , 90.887764, 90.887764, 90.887764,
           93.748886, 90.887764, 91.87696 , 90.887764, 90.887764,
           93.07386 , 90.887764, 90.887764, 90.887764, 90.887764,
           90.887764, 93.98111 , 90.931885, 90.887764, 90.887764,
           91.122955],
          [98.57535 , 95.99892 , 90.887764, 90.887764, 90.887764,
           90.887764, 90.887764, 94.597786, 90.887764, 90.887764,
       

## Convert prediction back to ```netcdf``` file

In [58]:
out = pred #np.zeros(shape=(1,1,1,21,21))
print(out.shape)
out = out.reshape(1,21,21)
print(out.shape)


(1, 1, 1, 21, 21)
(1, 21, 21)


In [61]:
data = xr.DataArray(data = out,
                     dims=('time', 'latitude', 'longitude',),
                     coords={'latitude': ds['latitude'].values,
                            'longitude': ds['longitude'].values,
                            'time': ds['t2m'].isel(time=slice(end-1, end))['time'].values,
                            })
data

<xarray.DataArray (time: 1, latitude: 21, longitude: 21)>
array([[[90.887764, 90.887764, 90.887764, 91.39598 , 91.51119 ,
         90.887764, 90.887764, 90.887764, 93.77147 , 90.887764,
         91.389565, 90.887764, 90.887764, 90.887764, 92.01836 ,
         90.887764, 90.887764, 90.887764, 91.58931 , 90.887764,
         90.887764],
        [90.887764, 90.887764, 91.75855 , 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 94.156   ,
         90.887764, 92.41626 , 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 92.00379 , 90.887764, 90.887764,
         90.887764],
        [99.02188 , 95.13313 , 90.887764, 90.887764, 90.887764,
         93.748886, 90.887764, 91.87696 , 90.887764, 90.887764,
         93.07386 , 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 93.98111 , 90.931885, 90.887764, 90.887764,
         91.122955],
        [98.57535 , 95.99892 , 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 94.597786, 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 91.76505 , 91.51585 ,
         90.887764],
        [90.887764, 90.887764, 90.887764, 90.887764, 90.89985 ,
         90.887764, 90.887764, 90.887764, 90.887764, 93.93312 ,
         90.887764, 96.58669 , 92.863396, 90.887764, 92.06667 ,
         90.887764, 90.887764, 91.02223 , 90.887764, 90.887764,
         90.887764],
        [92.426476, 90.887764, 91.05199 , 90.887764, 95.45771 ,
         90.887764, 91.505264, 91.34152 , 90.887764, 91.97836 ,
         91.60448 , 90.887764, 90.887764, 93.18929 , 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764],
        [90.887764, 90.887764, 90.887764, 92.47607 , 94.11214 ,
         90.887764, 90.887764, 92.678764, 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 96.28652 , 91.42509 , 97.84587 , 91.689285,
         90.887764],
        [90.887764, 90.887764, 90.887764, 92.021736, 90.887764,
         90.887764, 92.27654 , 90.887764, 91.0986  , 90.887764,
         90.887764, 90.887764, 91.660446, 90.887764, 90.887764,
         95.26064 , 90.887764, 90.887764, 90.887764, 92.59237 ,
         90.887764],
        [90.887764, 90.887764, 90.887764, 92.92827 , 94.13754 ,
         90.887764, 91.49754 , 90.887764, 93.35654 , 91.95608 ,
         93.74119 , 93.209656, 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764],
        [91.11772 , 90.887764, 95.54082 , 90.887764, 90.887764,
         91.16124 , 90.887764, 90.887764, 90.887764, 92.922066,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 96.209656, 90.887764, 90.887764, 90.887764,
         92.53378 ],
        [90.887764, 90.887764, 92.76047 , 91.384   , 90.887764,
         90.887764, 93.149765, 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 91.64251 , 90.887764, 91.91278 ,
         90.887764, 96.46129 , 91.44396 , 90.887764, 90.887764,
         91.512314],
        [90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 96.258255, 90.887764, 92.14034 , 90.887764,
         93.34398 , 90.887764, 92.10625 , 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764],
        [90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 94.08852 , 91.55054 , 97.208755, 90.887764,
         92.63596 , 90.887764, 96.96939 , 91.54877 , 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764],
        [90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 94.09282 , 90.887764,
         90.887764, 91.548195, 96.52736 , 90.887764, 90.887764,
         90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
         90.887764],
        [90.887764, 90.887764, 90.887764, 90.887764, 90.887764,
